
# Debugging in Python

Everyone makes mistakes in their code — even experienced programmers. The key is learning how to **read error messages**, **identify bugs**, and **fix them efficiently**. This notebook introduces strategies and tools to help you do just that.

---

## Why Debugging Matters

- Python will often tell you exactly what went wrong — if you know how to read the message.
- Common bugs often come from scope issues, typos, wrong assumptions, or misuse of data structures. 
- Most of the time, you will get error messages because of simple mistakes such as typos or incorrect function calls.

---

## Anatomy of an Error Message

Let’s look at a typical Python error and break it down line by line:



In [1]:
print(5 + "5")

TypeError: unsupported operand type(s) for +: 'int' and 'str'

### What does this mean?

- `Traceback`: Shows the call stack — the sequence of code that led to the error.
- `Cell In[1]`: This error occurred in cell 1 of the notebook.
- `line 1`: The exact line that caused the error.
- `print(5 + "5")`: The code that failed — trying to add an integer and a string.
- `TypeError`: The type of error. This one is about mixing incompatible types.
- `unsupported operand type(s)`: Tells you why it failed — `'int' + 'str'` is not allowed.

### Tip:
Always start reading from the **bottom** of the error message. That’s where Python tells you what actually went wrong.

**Video:** [Why is a programming error called a bug?](https://www.youtube.com/watch?v=rhFSG-VyR_E)

**General advice:**
Code is always partly a **black box**, thus **print and plot results** to convince yourself (and others) that your results are sensible. You can also use  `assert` statements to check that your assumptions are correct. If they are not, the program will stop and give you an error message.

In [2]:
# Example: Finding the first even number in a list
numbers = [1, 3, 5, 7, 8, 10]

for i, num in enumerate(numbers):
    print(f"Index: {i}, Number: {num}")  # Debugging: Print the current index and number
    if num % 2 == 0:
        print(f"Found an even number: {num} at index {i}")
        break

Index: 0, Number: 1
Index: 1, Number: 3
Index: 2, Number: 5
Index: 3, Number: 7
Index: 4, Number: 8
Found an even number: 8 at index 4


**Assertions:** You can enforce assertations on e.g. numeric values

In [ ]:
# Goal: Take a number to the power of 2 (Python command: **)
x = -2
y = x*2 # forgot one *
assert y > 0, f'x = {x}, y = {y}' # if y is not greater than 0, raise an AssertionError with the message

AssertionError: x = -2, y = -4

**Exceptions:** When code fails, it generates (*raises*) an exception. You can catch exceptions and handle them. This is useful for debugging and for writing robust code that can handle unexpected situations.

In [5]:
x = 0.0
try:
    if x < 0.0:
        raise ValueError('x cannot be negative')
    else:
        print('successful')
except Exception as e:
    print(e)

successful


Consider the following code:

In [4]:
a = 0.8
xlist = [2,3,-1]

def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        y += z
    return y

y = myfun(xlist,a)
print(y)

(3.340308817497993+0.5877852522924732j)


**Problem:** Our result is a complex number. We did not expect that. Why does this problem arise?

**Find the error with print:**

In [7]:
def myfun(xlist,a):
    print(f'a = {a}') 
    y = 0
    for x in xlist:
        z = x**a
        print(f'y(old) = {y}; x = {x} -> y(new) = y(old) + x^a = {z}') # temp
        y += z
    return y

myfun(xlist,a)

a = 0.8
y(old) = 0; x = 2 -> y(new) = y(old) + x^a = 1.7411011265922482
y(old) = 1.7411011265922482; x = 3 -> y(new) = y(old) + x^a = 2.4082246852806923
y(old) = 4.149325811872941; x = -1 -> y(new) = y(old) + x^a = (-0.8090169943749473+0.5877852522924732j)


(3.340308817497993+0.5877852522924732j)

**Solution with an assert:**

In [10]:
import numpy as np

def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        assert np.isreal(z), f'z is not real for x = {x}, with z={z}'
        y += z
    return y
    
try:
    myfun(xlist,a)
except Exception as e:
    print(e)

z is not real for x = -1, with z=(-0.8090169943749473+0.5877852522924732j)


**Solution with if and raise exception:**

In [11]:
def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        if not np.isreal(z):
            print(f'z is not real for x = {x}, with {z}')
            raise ValueError('negative input number') # an exception will be raised here  
        y += z
    return y

try:
    myfun(xlist,a)
except Exception as e:
    # we'll end up down here because the exception was raised. 
    print(e)

z is not real for x = -1, with (-0.8090169943749473+0.5877852522924732j)
negative input number


**Note:** You could also decide that the function should return e.g. **nan** when experiencing a complex number.

In [12]:
def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        if not np.isreal(z):
            return np.nan
        y += z
    return y

myfun(xlist,a)

nan


### ⚠️ NumPy Warnings

NumPy is a powerful library for numerical computation, but like all tools, it can behave unexpectedly if used carelessly.

A common source of confusion is **dividing by zero**, which doesn't raise a standard error but gives a **warning** and produces `nan` (not a number) or `inf` (infinity):


These warnings are **easy to overlook**, so always double-check your results, especially when dealing with divisions or logs.

Use `np.isnan()` or `np.isinf()` to detect problematic values.


Here we see an example of a *RuntimeWarning*.

In [14]:
xlist = [-1,2,3]
def f(xlist):
    y = np.empty(len(xlist))
    for i,x in enumerate(xlist):
        y[i] = np.log(x)
    return y

log_xlist = f(xlist)
print(log_xlist)

assert np.all(np.isnan(log_xlist)), 'Not all values are NaN' # check if any of the values are NaN
assert sum(np.isnan(log_xlist)) == 0, 'Not all values are NaN' # check if any of the values are NaN by counting them and asserting the count is 0

[       nan 0.69314718 1.09861229]


C:\Users\Admin2\AppData\Local\Temp\ipykernel_35824\3179641222.py:5: RuntimeWarning: invalid value encountered in log
  y[i] = np.log(x)


AssertionError: 


### 🧠 Scope Bugs

One of the most common beginner mistakes is **using a variable outside of its scope**.

Variables defined inside a function are **local** — they don't exist outside of it.




In [22]:
def calculate():
    test = 10
    return test

print(test)  # NameError: x is not defined

NameError: name 'test' is not defined

In this case, `x` only exists inside `calculate()`. To fix this, return the value and assign it outside:

**Tip**: If you see a `NameError`, always check whether you're trying to use a variable that hasn’t been defined in the current scope.

In [23]:
result = calculate()
print(result)

10


Global variables are dangerous: they can lead to code that is hard to debug and maintain. It's better to use function parameters and return values to pass data around.

In [24]:
# a. define a function to multiply a variable with 5
a = 5
def f(x):
    return a*x

# many lines of code
# many lines of code
# many lines of code

# z. setup the input and call f
y = np.array([3,3])
a = np.mean(y)
b = np.mean(f(y))   # a = 5 was overwritten by the last line

print(b)

9.0


**Conclusion:** 

1. Never use global variables, they can give poisonous **side effects**.
2. Use a *positional argument* or a *keyword argument* instead. 


### 🔢 Index Errors

Python uses **zero-based indexing**, which means the first element is at index 0.

Trying to access an index that doesn’t exist will raise an `IndexError`:


In [25]:
lst = [10, 20, 30]
print(lst[3])  # IndexError: list index out of range

IndexError: list index out of range

This happens often when:
- You iterate too far in a loop
- You forget that the last valid index is `len(list) - 1`
- You assume the shape of a list or array without checking

**Tip**: Use `len()` to check how many elements are in a list before accessing them.

In [28]:
# a. setup
N = 10
x = np.linspace(1.3,8.2,N)

i = 0
for i in range(len(x)):
    i += 1
    print(x[i])

2.0666666666666664
2.833333333333333
3.5999999999999996
4.366666666666666
5.133333333333333
5.8999999999999995
6.666666666666666
7.433333333333333
8.2


IndexError: index 10 is out of bounds for axis 0 with size 10

**Task:** Solve the error.

## 5. <a id='toc5_'></a>[Debugger](#toc0_)

### 5.1. <a id='toc5_1_'></a>[In notebooks](#toc0_)

Consider this example:

In [29]:
def f(x):
    y = x-5
    z = x
    return np.log(y*z)

In [30]:
x = 4
q = f(x)
print(q)

C:\Users\Admin2\AppData\Local\Temp\ipykernel_31100\529491813.py:4: RuntimeWarning: invalid value encountered in log
  return np.log(y*z)


nan


**Task:** Let us analyze why the problem arises with the *debugger*.

1. Go to first line of `f(x)` two cells above. Press `F9` to create a *breakpoint*
2. Go to the cell just above this cell. Press `Ctrl+Shift+Alt+Enter` to run the cell in debug mode. 
   - This will run the cell and stop at the breakpoint you set in the function `f(x)`.
3. Press `F10` to `Step Over`. Notice the value of y?
4. Exit with `Shift+F5` or continue with `F5` till the code has run through.

**Extra:** Place the breakpoint at `x = 4`. Try out `F11` to `Step Into` the function `f(x)` and `Shift+F11` to `Step Out`

**More details:** See [here](https://code.visualstudio.com/docs/datascience/jupyter-notebooks#_debug-cell).

### 5.2. <a id='toc5_2_'></a>[In modules](#toc0_)

We can also use the debugger if we execute code from modules.

In [1]:
# These are IPython magic commands used in Jupyter notebooks to automatically reload modules when they are modified. 
%load_ext autoreload
%autoreload 2

Load in the module and set a breakpoint in the module. Then execute the code in debug mode.

In [2]:
# This is a Python script that imports a module named 'mymodule'.
# Ensure that the module is in the same directory or in the Python path.
import mymodule

In [5]:
x = 4
q = mymodule.g(x)
print(q)

c:\Users\Admin2\Meine Ablage\Uni\PhD_Tuebingen\Teaching\E383\E383_SoSe25\Code\mymodule.py:6: RuntimeWarning: invalid value encountered in log
  return np.log(y*z)


nan
